In [133]:
import yfinance as yf
import os 
import pandas as pd 
import numpy as np
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime


In [134]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_table = pd.read_html(url)[0]
# to download S&P500 data, set ticker_list = sp500_tickers
sp500_tickers = sp500_table['Symbol'].tolist()
ticker_list = ["JPM"]


api_key = os.getenv("ALPACA_API_KEY")
secret_api_key = os.getenv("ALPACA_SECRET_KEY")
client = StockHistoricalDataClient(api_key, secret_api_key)


In [135]:
total_data = []
folder = "data"
data_type = input("""Would you like the data tobe from Alpaca or YF?
                  Alpaca - 1
                  YF - 2
                  Note: Alpaca data requires env vars.""")
while data_type not in ["1", "2"]:
    print("input neither 1 or 2.")
    data_type = input("""Would you like the data tobe from Alpaca or YF?
                  Alpaca - 1
                  YF - 2
                  Note: Alpaca data requires env vars.""")
if data_type == "1":
    data_type = "Alpaca"
else:
    data_type = "YF"
for ticker in ticker_list:
    file = f"{ticker} {data_type} Data.parquet"
    file_path = os.path.join(folder, file)
    if os.path.exists(file_path):
        print(f"file for {ticker} already exists. Skipping...")
        continue
    try:
        if data_type == "YF":
            df = yf.download(ticker, period="max", progress=False)
            if df.empty:
                print(f"Downloaded data for {ticker} was completely empty. Skipping...")
                continue
            elif len(df) < 200:
                print(f"{df} only has {len(df)}")
                continue
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.get_level_values(0)
            if isinstance(df.index, pd.MultiIndex):
                df.reset_index(inplace=True)
                df.set_index("Date", inplace=True)
        elif data_type == "Alpaca":
            max_timeframe = StockBarsRequest(
                    symbol_or_symbols=[ticker],
                    timeframe=TimeFrame.Day,
                    start=datetime(1999, 1, 1),
                    end=datetime.now(),
                    )
            max_timeframe = client.get_stock_bars(max_timeframe).df
            if max_timeframe.index.name != "timestamp":
                if "timestamp" in max_timeframe.columns:
                    max_timeframe.set_index("timestamp", inplace=True)
            df = max_timeframe
        if df.empty:
            print(f"Downloaded data for {ticker} was completely empty. Skipping...")
            continue
        elif len(df) < 200:
            print(f"{df} only has {len(df)}")
            continue
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        if isinstance(df.index, pd.MultiIndex):
            df.reset_index(inplace=True)
            df.set_index("timestamp", inplace=True)

    except Exception as e:
        print(f"error, {e}")
    print(df)
    df.to_parquet(file)
    print(f"{ticker} done")


input neither 1 or 2.
input neither 1 or 2.
input neither 1 or 2.
input neither 1 or 2.
input neither 1 or 2.
input neither 1 or 2.
input neither 1 or 2.
input neither 1 or 2.
input neither 1 or 2.
input neither 1 or 2.
                          symbol    open      high      low   close  \
timestamp                                                             
2016-01-04 05:00:00+00:00    JPM   63.95   64.0600   63.010   63.62   
2016-01-05 05:00:00+00:00    JPM   63.69   64.1300   63.040   63.73   
2016-01-06 05:00:00+00:00    JPM   62.70   63.1250   62.340   62.81   
2016-01-07 05:00:00+00:00    JPM   61.46   62.0000   60.080   60.27   
2016-01-08 05:00:00+00:00    JPM   61.13   61.2700   58.850   58.92   
...                          ...     ...       ...      ...     ...   
2025-07-21 04:00:00+00:00    JPM  291.00  294.1800  290.580  290.97   
2025-07-22 04:00:00+00:00    JPM  291.50  293.6250  289.180  291.43   
2025-07-23 04:00:00+00:00    JPM  292.84  296.9900  292.550  296.76   